# Very simple agent

This notebook show how to create a very simple conversation agent, with no tools.

### Let's start by importing the necessary libraries and set the api keys

In [1]:
from dotenv import load_dotenv

from typing import List
from agente.core.base import BaseAgent

## Load and set environment variables from .env file
load_dotenv()

True

### This first example show how to create an agent without streaming.

We note that even the agent is set to run without stream, we need to iterate over agent.run() to get the messages. 
This allows to stream in case a second agent with stream is called by the first parent agent.

In [2]:
class VerySimpleAgent(BaseAgent):
    agent_name: str = "VerySimpleAgent"
    system_prompt:str = "You are a helpful AI assistant."
    completion_kwargs: dict = {
        "model": "claude-sonnet-4-20250514",
        "stream": False,
        "temperature": 1.0,
        "max_tokens": 5000,
        "reasoning_effort":"medium"
        # "thinking":{"type": "enabled", "budget_tokens": 1024}
    }
    
agent = VerySimpleAgent()
agent.add_message("user", "Please write me a funny joke.")

In [3]:
responses = await agent.run()
responses

Executing agent: VerySimpleAgent


[Response(call_id='chatcmpl-19c2dd33-7700-4156-a84a-b60b82da5045', agent_name='VerySimpleAgent', role='assistant', content="Why don't scientists trust atoms?\n\nBecause they make up everything! \n\n😄", tool_calls=[], reasoning_content="I should write a clean, family-friendly joke that's likely to get a chuckle. Let me think of something with a good setup and punchline structure.\n\nHere's one about a library that I think works well:", thinking_blocks=[ThinkingBlock(type='thinking', thinking="I should write a clean, family-friendly joke that's likely to get a chuckle. Let me think of something with a good setup and punchline structure.\n\nHere's one about a library that I think works well:", signature='Eu8CCkYIBBgCKkDanHhihbjgQqPzXM4WNFzKsW4yeGmrIdeF30tl9nxTM67sqwwQeifeV/WX/JSpoVN1WStKSOVWwuWlgS1qPjvrEgx5/rcdfnrPcleUqJcaDPQBw9HLYqumNqKvjiIw/YeZB8enHF17nwdMMDhKb+phOLaBvwMy6nN3plRKKv6ev85NMsuDObqgSExFwpz9KtYB/foJgKRj5LcYdMeAz5hvJKLoIFdV70ku8CSs4m0gOARuuQNnZzJPiUyRyqo2G9WF5fthJCezPvBf57o2U

### The conversation history is stored as well:

In [4]:
agent.conv_history.messages

[Message(role='system', agent_name='VerySimpleAgent', content=[Content(type='text', text='You are a helpful AI assistant.')], tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None, timestamp=datetime.datetime(2025, 6, 3, 13, 29, 41, 233720)),
 Message(role='user', agent_name='VerySimpleAgent', content=[Content(type='text', text='Please write me a funny joke.')], tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None, timestamp=datetime.datetime(2025, 6, 3, 13, 29, 41, 233809)),
 Message(role='assistant', agent_name='VerySimpleAgent', content=[ContentThinking(type='thinking', text="I should write a clean, family-friendly joke that's likely to get a chuckle. Let me think of something with a good setup and punchline structure.\n\nHere's one about a library that I think works well:", signature='Eu8CCkYIBBgCKkDanHhihbjgQqPzXM4WNFzKsW4yeGmrIdeF30tl9nxTM67sqwwQeifeV/WX/JSpoVN1WStKSOVWwuWlgS1qPjvrEgx5/rcdfnrPcleUqJcaDPQBw9HLYqumNqK

In [5]:
agent.add_message("user", "Another one.")
responses = await agent.run()
responses

Executing agent: VerySimpleAgent


[Response(call_id='chatcmpl-f9ad32e8-379c-4f4c-b79f-98effbcb08c4', agent_name='VerySimpleAgent', role='assistant', content='Why did the scarecrow win an award?\n\nBecause he was outstanding in his field!\n\n🌾😂', tool_calls=[], reasoning_content="The user wants another joke. I'll give them a different type of joke this time - maybe a pun or play on words that's different from the science joke I just told.", thinking_blocks=[ThinkingBlock(type='thinking', thinking="The user wants another joke. I'll give them a different type of joke this time - maybe a pun or play on words that's different from the science joke I just told.", signature='EsoCCkYIBBgCKkDuBktuEECX5/nC7lCDXFGXuLKAVoXIjvnIz4tj0eKC2pKoJykiHXQILC6aXUUXHY9ioHCJM+XJPuC9KWNh/lNkEgyJjq3QVo2VWO7nf+IaDHmNUpxRW44PTS6Z1yIwShsRo9pKZh0PfmOOdBGaheXGhtk04MQaA/YgTbNQNtDaZyr1mZ0S1QqhQRrNzSltKrEBUCEcCnN1eSpo+5XssuamCAaxBWnvJC5s5i0QRb3UGtS13hEZm7wtiz18K2vezRrAmfmowly6d747FTk35wHIIbM5GtU2WwouXQpieZrA7ByBNgCHYg7XabRQRES435NP5en0qY00BB4PQx5rLR1DO

## Streaming

### Now let's create the same agent but with stream. For that we can use Gradio for better visualization

In [11]:
import gradio as gr

class VerySimpleAgent(BaseAgent):
    agent_name: str = "VerySimpleAgent"
    completion_kwargs: dict = {
        "model": "gpt-4o",
        "stream": True,
        "temperature": 1.0,
        "max_tokens": 500,
    }


def get_new_agent():
    """Create a fresh agent instance"""
    new_agent = VerySimpleAgent()
    return new_agent

with gr.Blocks() as demo:
    chatbot = gr.Chatbot(type="messages")
    msg = gr.Textbox()
    clear = gr.Button("Clear")

    # Initialize with a function call instead of direct instantiation
    main_agent = gr.State(value=None)

    def user(user_message, agent):
        if agent is None:
            agent = get_new_agent()
        agent.add_message("user", user_message)
        return "", [{"role": "user", "content": user_message}], agent

    async def bot(history, agent):
        if agent is None:
            agent = get_new_agent()
        if not history:
            yield [], agent
            return
        
        history.append({"role": "assistant", "content": ""})
        response_generator = await agent.run(stream=True)
        async for chunk in response_generator:
            if chunk.content:
                history[-1]["content"] += chunk.content
                yield history, agent

    def reset_state():
        return None, get_new_agent()

    msg.submit(user, [msg, main_agent], [msg, chatbot, main_agent]).then(
        bot, [chatbot, main_agent], [chatbot, main_agent]
    )
    clear.click(reset_state, None, [chatbot, main_agent])

demo.launch()

/Users/miguelwon/miniconda3/envs/agente/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.


Executing agent: VerySimpleAgent
Executing agent: VerySimpleAgent


In [8]:
from agente.core.decorators import function_tool
import random

In [35]:

class MainAgent(BaseAgent):
    agent_name: str = "main_agent"
    
    # @function_tool 
    # def random_topic(self):
    #     """Tool to get a random topic.
    #     """
    #     topics = ["programming","science","animals","food","sports"]
    #     topic = random.choice(topics)

    #     return topic

In [36]:
main_agent = MainAgent()
main_agent.completion_kwargs["model"] = "claude-sonnet-4-20250514"
main_agent.completion_kwargs["reasoning_effort"] = "medium"
main_agent.completion_kwargs["temperature"] = 1.0
main_agent.add_message(role = "user", content = "Generate a random topic.")
responses = await main_agent.run(max_retries=10)

Executing agent: main_agent


In [37]:
main_agent.conv_history.messages

[Message(role='user', agent_name='main_agent', content=[Content(type='text', text='Generate a random topic.')], tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None, timestamp=datetime.datetime(2025, 6, 3, 16, 54, 51, 674741)),
 Message(role='assistant', agent_name='main_agent', content=[ContentThinking(type='thinking', text="I should generate a random topic that could be interesting to discuss. Let me think of something that's neither too mundane nor too specialized - something that could spark conversation or curiosity.\n\nSome options:\n- The psychology of why people collect things\n- How different cultures view time\n- The history of street food\n- Why some songs get stuck in our heads\n- The art of letter writing in the digital age\n- Urban wildlife adaptation\n- The science behind why we yawn\n\nI'll go with one that's both relatable and has some depth to explore.", signature='EsgFCkYIBBgCKkBrPKvLcWAutjUgR5QYo7G1K4V8wBkWl31/wSEvcMw4OXSRMqqw+tPh1Ud

In [38]:
main_agent.conv_history.messages[0].dict()

/var/folders/pp/vhr348yx1_vd7j7p3b6gf5gr0000gn/T/ipykernel_38787/800015913.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  main_agent.conv_history.messages[0].dict()


{'role': 'user',
 'agent_name': 'main_agent',
 'content': [{'type': 'text', 'text': 'Generate a random topic.'}],
 'tool_calls': None,
 'tool_call_id': None,
 'tool_name': None,
 'hidden': False,
 'id': None,
 'usage': None,
 'timestamp': datetime.datetime(2025, 6, 3, 16, 54, 51, 674741)}

In [39]:
main_agent.conv_history.messages[1].dict()

/var/folders/pp/vhr348yx1_vd7j7p3b6gf5gr0000gn/T/ipykernel_38787/3833547919.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  main_agent.conv_history.messages[1].dict()


{'role': 'assistant',
 'agent_name': 'main_agent',
 'content': [{'type': 'thinking',
   'text': "I should generate a random topic that could be interesting to discuss. Let me think of something that's neither too mundane nor too specialized - something that could spark conversation or curiosity.\n\nSome options:\n- The psychology of why people collect things\n- How different cultures view time\n- The history of street food\n- Why some songs get stuck in our heads\n- The art of letter writing in the digital age\n- Urban wildlife adaptation\n- The science behind why we yawn\n\nI'll go with one that's both relatable and has some depth to explore.",
   'signature': 'EsgFCkYIBBgCKkBrPKvLcWAutjUgR5QYo7G1K4V8wBkWl31/wSEvcMw4OXSRMqqw+tPh1UdImXGUu2xn1owwj0jcNPe/4tW64xzZEgxGvTNuPSfZefw3chIaDLhSZSRjga7wbSTF9SIw4EFjlmyQ0Wi+sh2O1qiYUA8T3397/gaxzKf27bnKnwK01+tMhHtEICL52gwPd+dJKq8Ed8tKgGGBXbfciYMyctbecXW9J8S0bj7PznL8/mk+3Wx+F3CsompwWzmQRQndDlaAZADeOKI/AFZx27WvUGstb3gnfYf7cTrbqSSqcPbGB8KeKHl4EwxaGeNYT

In [40]:
main_agent.add_message(role = "user", content = "Another one.")
responses = await main_agent.run(max_retries=10)

Executing agent: main_agent


In [42]:
main_agent.conv_history.messages[3].dict()

/var/folders/pp/vhr348yx1_vd7j7p3b6gf5gr0000gn/T/ipykernel_38787/863773376.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  main_agent.conv_history.messages[3].dict()


{'role': 'assistant',
 'agent_name': 'main_agent',
 'content': [{'type': 'thinking',
   'text': "I should generate another random topic that's different from the first one. Let me think of something completely unrelated - maybe something more concrete or creative.",
   'signature': 'EtACCkYIBBgCKkBH/MovmRFZd5icESBImq+W8Ptf/2qy25ADWHh41P1nxib6Saxjq1A2hOkUr3Fj7uc/qeipBvgY2FyhBQ4xcU2xEgx3n+nInTdhK8aMNQYaDA4PeDVTPFFr42t3pSIwTJ+7ZDfp0KOblIb6EIuj4OyBgU14IMtNM4sq2J0Ap2PtX+U05aMzgvn1WAHJGoT9KrcBASl1s6YKKkfF068rmsuOvEtJUdkdtLVPAoLEahCEFmG1BBo+f4Y1SG7IMyNZiPmuaKtSn4XhaJsNA2cFx6aEwY2Xia76tiwDshPV5TeW6XiVZeSClrIhwPg3DCUQwqoJsPt/hwCwJQYifliDFAmi66474/URt55XOEuZf7+13JcSvlMTjMTY4hf2k+kLYnlx7umkyPiSIeMWDZa1+wyDibdQzTVwP73QUVpIScZdTRAMKmE5KiDDGAE='},
  {'type': 'text',
   'text': "**The secret lives of library books**\n\nThe fascinating journey of a single book through decades of different readers, the marginalia left behind, the places it's traveled, the homes it's lived in, and all the stories it cou

In [26]:
main_agent.conv_history.messages[3].dict()

/var/folders/pp/vhr348yx1_vd7j7p3b6gf5gr0000gn/T/ipykernel_38787/863773376.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  main_agent.conv_history.messages[3].dict()


{'role': 'tool',
 'agent_name': 'main_agent',
 'content': [{'type': 'text', 'text': '"animals"'}],
 'tool_calls': None,
 'tool_call_id': 'toolu_01PF5764MMEK1m85R8R6agzJ',
 'tool_name': None,
 'hidden': False,
 'id': None,
 'usage': None,
 'timestamp': datetime.datetime(2025, 6, 3, 15, 23, 7, 876146)}

In [27]:
main_agent.conv_history.messages[4].dict()

/var/folders/pp/vhr348yx1_vd7j7p3b6gf5gr0000gn/T/ipykernel_38787/3224363010.py:1: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  main_agent.conv_history.messages[4].dict()


{'role': 'assistant',
 'agent_name': 'main_agent',
 'content': [{'type': 'text',
   'text': 'Your random topic is: **Animals**\n\nThis is a broad and fascinating topic that could cover everything from wildlife conservation and animal behavior to pets, endangered species, marine life, or the role of animals in different ecosystems. What would you like to explore about animals?'}],
 'tool_calls': None,
 'tool_call_id': None,
 'tool_name': None,
 'hidden': False,
 'id': None,
 'usage': {'completion_tokens': 57, 'prompt_tokens': 425, 'total_tokens': 482},
 'timestamp': datetime.datetime(2025, 6, 3, 15, 23, 10, 270382)}

In [20]:
main_agent.add_message(role = "user", content = "Another one.")
responses = await main_agent.run(max_retries=10)

Executing agent: main_agent
Executing tool: random_topic (agent: main_agent)
Executing agent: main_agent


In [21]:
main_agent.conv_history.messages

[Message(role='user', agent_name='main_agent', content=[Content(type='text', text='Generate a random topic.')], tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None, timestamp=datetime.datetime(2025, 6, 3, 15, 23, 5, 486742)),
 Message(role='assistant', agent_name='main_agent', content=[Content(type='text', text="I'll generate a random topic for you.")], tool_calls=None, tool_call_id=None, tool_name=None, hidden=False, id=None, usage=Usage(completion_tokens=46, prompt_tokens=365, total_tokens=411), timestamp=datetime.datetime(2025, 6, 3, 15, 23, 7, 875725)),
 Message(role='assistant', agent_name='main_agent', content=[], tool_calls=[{'index': 1, 'function': {'arguments': '{}', 'name': 'random_topic'}, 'id': 'toolu_01PF5764MMEK1m85R8R6agzJ', 'type': 'function'}], tool_call_id=None, tool_name=None, hidden=False, id=None, usage=None, timestamp=datetime.datetime(2025, 6, 3, 15, 23, 7, 875756)),
 Message(role='tool', agent_name='main_agent', content=[Content